In [2]:
import sys, os
sys.path.append(os.path.abspath("/Users/DavidHolzwarth/Uni/EPFL/bachelors-thesis"))

import time
import numpy as np
from LP import resolve_delegations_LP, invert_graph, set_up_LP, solve_LP
from graph_viz import visualize_delegation_graph
from iterative import iterate_delegations
import matplotlib.pyplot as plt
import json

np.random.seed(1)

WEIGHTS = [(i + 1) / 10 for i in range(1, 10)]
NUM_NODES = 5
NUM_LOOPS = 0

def get_random_delegation_weights(n: int) -> list:
    """
    Generates at most `n` random weights from the set of {0.1, 0.2, ..., 0.9} such that they sum to exactly 1.0.

    Parameters:
    - n (int): The number of weights to generate.

    Returns:
    - list[float]: A list of `n` values that sum to 1.0.

    Notes:
        - If `n == 0`, an empty list is returned.
        - The algorithm may find less than 'n' weights, if e.g. n = 2, but the first weight is 1.0, then it will only return [1.0]
        - The last weight is explicitly adjusted to correct any rounding errors.
        - The algorithm ensures no negative values by rejecting choices that exceed the remaining sum.
    """
    if n == 0: return []

    diff = 1
    weights = []
    for i in range(n-1):
        for j in range(11):
            # Tries 10 times to find a valid weight. If this is not possible the algorithm gives up
            choice = np.random.choice(WEIGHTS)
            if diff - choice > 0:
                weights += [choice]
                diff -= weights[-1]
                break
    
    # If we have not used all the weights, add the last one
    if diff > 0:
        weights += [round(diff, 1)]

    return weights

def is_connected_to_sink(delegations, start_node):
    visited = set()

    def dfs(node):
        if node in visited:
            return False 
        
        visited.add(node)

        # A sink is a node with no outgoing edges 
        if node not in delegations or not delegations[node]:  
            return True  

        # Recur for all outgoing connections
        return any(dfs(neighbor) for neighbor in delegations[node])

    return dfs(start_node)

def create_delegation_graph(num_nodes, num_loops):

    nodes = []
    delegations = {}
    for i in range(num_nodes):
        node = str(i)
        nodes += [node]
        # Choose how many delegations this node shall have
        num_delegations = np.random.randint(0, min(3, i) + 1)
        if num_delegations > 0:
            delegation_weights = get_random_delegation_weights(num_delegations)

            delegates = np.random.choice(nodes, num_delegations, replace=False)
            # Sorting assures that if the node delegates to itself, this delegation is first in the list.
            delegates = sorted(delegates, reverse=True) 

            for j in range(len(delegation_weights)):
                delegate = delegates[j]

                # Avoids delegations of N -1-> N, which are not allowed (since N is not voting and voting at the same time)
                if (delegate == node) and ((delegation_weights[j] == 1) or delegations.get(node, {}).get(node, 0) + delegation_weights[j] == 1):
                    # If the only delegate is itself, turn this node into a sink by just not adding the delegation 
                    if len(delegates) == 1:
                        break 
                    # Else remove the delegation, and add the weight to the next delegate's weight
                    else:
                        delegation_weights[j + 1] +=  delegation_weights[j] 
                        continue

                else:
                    delegations.setdefault(node, {}).setdefault(delegate, 0)
                    delegations[node][delegate] += delegation_weights[j]

    for i in range(num_loops):
        
        # Try to find a non-sink node with which to create a loop
        node = None
        for j in range(len(nodes)):
            node = np.random.choice(nodes)
            if node not in delegations or len(delegations[node]) == 0:
                node = None
            else:
                break

        for j, delegate in enumerate(delegations.get(node, {})):
            # Try to create a loop between this node and the selected delegate

            # #If delegate is a sink
            if delegate not in delegations or len(delegations[delegate]) == 0:
                # Add the loop
                delegations[delegate] = {node: 1}

                # Check that we did not create an illegal cycle
                if not is_connected_to_sink(delegations, node):
                    # Remove the loop
                    del delegations[delegate]
                    continue
            # Delegate is not a sink
            else:

                # Choose any weight from the set of weights, except 1
                # Adding a new delegation with weight 1 might create a cycle
                new_delegation_weight = np.random.choice([weight for weight in WEIGHTS if weight not in [1]])

                # Free up weight from the other delegations
                for k, v in delegations[delegate].items():
                        delegations[delegate][k] = (1 - new_delegation_weight) * v

                # Add this weight directed toward the node, to create a loop 
                delegations[delegate].setdefault(node, 0)
                delegations[delegate][node] += new_delegation_weight

                break

    return delegations, nodes

/opt/anaconda3/envs/thesis/lib/python3.12/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
delegations, nodes = create_delegation_graph(10, 2)

model, _ = set_up_LP(invert_graph(delegations), nodes)

#%timeit solve_LP(model)
#%timeit iterate_delegations(invert_graph(delegations), nodes)
powers, sink_nodes = resolve_delegations_LP(invert_graph(delegations), nodes)

print(powers)

powers = {node: power if node in sink_nodes else 0 for node, power in powers.items()}
relevant_powers = [(node, powers[node]) for node in nodes if node in sink_nodes]
display(relevant_powers)
visualize_delegation_graph(delegations, powers)

{'0': 3.0, '1': 2.0, '2': 1.4333333, '3': 4.5666667, '4': 4.1666667, '5': 1.0, '6': 1.0, '7': 2.0, '8': 1.0, '9': 1.0, '__dummy': None}


[('0', 3.0), ('2', 1.4333333), ('3', 4.5666667), ('5', 1.0)]

Parsing the data...Done


No trigger


DelegationResolution:
MINIMIZE
None
SUBJECT TO
Constraint_1: 1 - 6 = 1

Constraint_9: 9 = 1

Constraint_4: 0.24 4 = 1

Constraint_6: 6 = 1

Constraint_8: 8 = 1

Constraint_7: 7 - 9 = 1

SinkNodesConstraint: 0 + 2 + 3 + 5 = 10

VARIABLES
0 free Continuous
1 free Continuous
2 free Continuous
3 free Continuous
4 free Continuous
5 free Continuous
6 free Continuous
7 free Continuous
8 free Continuous
9 free Continuous

# Benchmarking

In [5]:
LOOPS = 2

range_x = range(0, 50, 5)

time_data_iterative = []
time_data_lp = []
for num_loops in range(1):
    times_lp = []
    times_iterative = []
    for num_nodes in range_x:
        print(num_nodes)
        delegations, nodes = create_delegation_graph(num_nodes, LOOPS)
        delegations= invert_graph(delegations)

        n = 1 
        if 0 <= num_nodes < 500:
            n = 100
        elif 500 <= num_nodes < 1000:
            n = 50

        # LP
        start_time = time.time()
        for _ in range(n):
            #f = 1
            try:
                powers, sinks = resolve_delegations_LP(delegations, nodes)
            except Exception as e:
                display(powers)
                visualize_delegation_graph(invert_graph(delegations), powers)
                raise Exception
        end_time = time.time()

        times_lp.append((end_time - start_time) / n)

        # Iterative
        start_time = time.time()
        for _ in range(n):
            p = iterate_delegations(delegations, nodes, 0.001)
        end_time = time.time()

        times_iterative.append((end_time - start_time) / n)

    time_data_lp.append(times_lp)
    time_data_iterative.append(times_iterative)

x = list(range_x)

plt.clf()

for i in range(len(time_data_lp)):
    plt.plot(x, time_data_lp[i], label=f"LP") 
    plt.plot(x, time_data_iterative[i], label=f"Iterative")  

plt.legend()
plt.xlabel("Amount of Nodes")
plt.ylabel("Time (s)")
plt.title("Runtime of Delegation Algorithms")
plt.show()

0
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
set()
[]
5
{'2', '1'}
['0', '3', '4']
{'2', '1'}
['0', '3', '4']
{'2', '1'}
['0', '3', '4']
{'2', '1'}
['0'

KeyboardInterrupt: 

In [ ]:
#  Nodes 0 -> 60 000, step 10 000


with open("../data/large_graph_no_loops.json", "r") as fd:
    y = json.load(fd)

plt.figure()
plt.plot(x, y, label=f"LP")
plt.legend()
plt.xlabel("Amount of Nodes")
plt.ylabel("Time (s)")
plt.title("Runtime of Delegation Algorithms")
plt.show()

plt.figure()
plt.yscale("log")
plt.plot(x, y, label=f"LP") 
plt.legend()
plt.xlabel("Amount of Nodes")
plt.ylabel("Time (s) (log scale)")
plt.title("Runtime of Delegation Algorithms (log scale)")
plt.show()

plt.legend()
plt.xlabel("Amount of Nodes")
plt.ylabel("Time (s)")
plt.title("Runtime of Delegation Algorithms")
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: '../data/large_graph_no_loops.json'